In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/unbiased-teacher-main

/content/drive/MyDrive/unbiased-teacher-main


In [ ]:
# !unrar x '/content/drive/MyDrive/unbiased-teacher-main/data_v19.rar'

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-331huxt9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-331huxt9
  Resolved https://github.com/facebookresearch/detectron2.git to commit 017abbfa5f2c2a2afa045200c2af9ccf2fc6227f
  Preparing metadata (setup.py) ... done


# 1. Register dataset

# 2. TRAIN

In [ ]:
%cd /content/drive/MyDrive/unbiased-teacher-main

/content/drive/MyDrive/unbiased-teacher-main


In [ ]:
!CUDA_LAUNCH_BLOCKING=1 python train_net.py --num-gpus 1 --config configs/coco_supervision/faster_rcnn_R_50_FPN_sup1_run1_custom.yaml

Command Line Args: Namespace(config_file='configs/coco_supervision/faster_rcnn_R_50_FPN_sup1_run1_custom.yaml', resume=False, eval_only=False, num_gpus=1, num_machines=1, machine_rank=0, dist_url='tcp://127.0.0.1:49152', opts=[])
[11/24 06:30:40 detectron2]: Rank of current process: 0. World size: 1
[11/24 06:30:42 detectron2]: Environment info:
-------------------------------  -----------------------------------------------------------------
sys.platform                     linux
Python                           3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
numpy                            1.23.5
detectron2                       0.6 @/usr/local/lib/python3.10/dist-packages/detectron2
Compiler                         GCC 11.4
CUDA compiler                    CUDA 11.8
detectron2 arch flags            7.5
DETECTRON2_ENV_MODULE            <not set>
PyTorch                          2.1.0+cu118 @/usr/local/lib/python3.10/dist-packages/torch
PyTorch debug build              False
torch

# 3.Inference

In [ ]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("TRAIN_DATASET", {}, "data_v19/coco/annotations/instances_train2017.json", "data_v19/coco/train")
register_coco_instances("VAL_DATASET", {}, "data_v19/coco/annotations/instances_val2017.json", "data_v19/coco/valid")

In [ ]:
import os
from detectron2.config import get_cfg

cfg = get_cfg()

cfg.merge_from_file("configs/Base-RCNN-FPN.yaml")
cfg.MODEL.WEIGHTS = os.path.join("output_20/model_best.pth")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50   # Testing threshold
cfg.DATASETS.TRAIN = ("TRAIN_DATASET", )
cfg.DATASETS.TEST = ("VAL_DATASET", )

In [ ]:
from detectron2.data import DatasetCatalog
import cv2, random
from google.colab.patches import cv2_imshow
from detectron2.utils.visualizer import Visualizer
from ubteacher.engine.trainer import CustomPredictor

dataset_dicts = DatasetCatalog.get("VAL_DATASET")
predictor = CustomPredictor(cfg)

for d in random.sample(dataset_dicts, 2):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   scale=1
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])
    #
    annotations = d.get("annotations", None)
    if annotations is not None:
        for annotation in annotations:
            label = annotation["category_id"]
            bbox = annotation["bbox"]
            x, y, w, h = [int(coord) for coord in bbox]
            cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 2)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            font_color = (255, 255, 255)
            cv2.putText(im, str(label), (x, y - 10), font, font_scale, font_color, 2)

        # Hiển thị hình ảnh gốc với bounding boxes và nhãn gốc
        cv2_imshow(im)

Output hidden; open in https://colab.research.google.com to view.